In [1]:
#this program will run the difference image analysis

#if you use this code, please cite Oelkers et al. 2015, AJ, 149, 50, Alard & Lupton 1998, Alard et al. 2000,  Miller et al. 2008 and Oelkers & Stassun 2018

#import the relevant libraries for basic tools
import numpy as np
import scipy
from scipy import stats
import scipy.ndimage as ndimage
import astropy
from astropy.stats import sigma_clipped_stats
import math
import time
from time import strftime
from photutils import aperture_photometry
from photutils import CircularAperture
from photutils import CircularAnnulus
import random

#for reading in fits files
from astropy.io import fits

#import relevant libraries for a list
import glob, os
from os import listdir
from os.path import isfile, join, exists

In [2]:
###UPDATE HERE#####
#compile the C differencing program --- you will need to change the directories where the cfitiso directory is kept
compdiff = os.system('gcc sub_teste.c -lcfitsio -lm -lcurl')


In [24]:
# Input here the season and skygroup

season = 3
skygroup = 1
skygroup = str(skygroup)
season = str(season)
sky = str(skygroup).zfill(2)

In [25]:
comp = 'usuario'

#useful directories
cdedir = '/home/'+comp+'/dia_kepler/mychanges/Python/' #code directory
caldir = '/home/'+comp+'/Documents/Mestrado/FFIs/testeall/clean/masters/season' # directory for the location of the master frame
clndir = '/home/'+comp+'/Documents/Mestrado/FFIs/testeall/clean/s'+season # directory of where the images are located
difdir = '/home/'+comp+'/Documents/Mestrado/FFIs/testeall/clean/s'+season+'/dif'+season+'/' # directory to put the differenced images

#the optimal aperture to use from refphot.py
rad = 2.25

#size of the kernel, stamp and if you want an order
krnl = 2
stmp = 3
ordr = 0
nrstars = 499

In [26]:
#read in the master frame

mast, mheader = fits.getdata(caldir+season+'/new'+sky+'.fits', header = True,ext=1)
mean, median, std = sigma_clipped_stats(mast, sigma = 3)
expm_time = fits.getval(caldir+season+'/'+'new'+sky+'.fits','EXPTIME',ext=1)

#subtract the background from the master frame
nmast = mast-median

#write the new master file
mhd = fits.PrimaryHDU(nmast, header=mheader)
mhd.writeto(cdedir+'ref.fits', overwrite = True)
expm_time = fits.getval(cdedir+'ref.fits', 'EXPTIME')*3600.*24.

#get the image list and the number of files which need reduction
os.chdir(clndir) #changes to the raw image direcotory
files = [f for f in glob.glob("*.fits") if isfile(join(clndir, f))] #gets the relevant files with the proper extension
files = np.sort(files)
nfiles = len(files)
os.chdir(cdedir) #changes back to the code directory

#read in the star list
ids, xx, yy = np.loadtxt(caldir+season+'/'+skygroup+'_starlist.txt', unpack = 1, delimiter = ',')
ids1, xm, ym, mflx, mflx_er = np.loadtxt(caldir+season+'/'+skygroup+'_master.flux', unpack = 1, usecols = (0,1,2,3,4), delimiter =',')

In [27]:
# files

In [28]:
# hld = files[0].split('.')
# finnme = hld[0]+'dxx.'+hld[1]

## Célula teste

In [29]:
for j in range(0,nfiles):
    hld = files[j].split('.')
    finnme = hld[0]+'dxx.'+hld[1]
    print(finnme)

    	#check to see if the differenced file already exists
#     if (os.path.isfile(difdir+finnme) == 0):
        #read in the image
    imglist = fits.open(clndir+'/'+files[j])
    for ii in range(1,len(imglist)):
        iheader = imglist[ii].header
        if str(iheader['SKYGROUP']) == skygroup:
            finnme = files[j]+'dxx.'+hld[1]
            img = imglist[ii].data #get the image info
            mean, median, std = sigma_clipped_stats(img, sigma = 3.0, iters = 5)

            nimg = img-median
            ihd = fits.PrimaryHDU(nimg, header=iheader)
            ihd.writeto(cdedir+'img.fits', overwrite = True)
            jd1 = fits.getval(cdedir+'img.fits','TSTART')
            jd2 = fits.getval(cdedir+'img.fits','TSTOP')
            exp_time = fits.getval(cdedir+'img.fits', 'EXPOSURE')*3600.*24.
            jd = np.mean([jd1,jd2])
            #print('Getting magnitudes from the star list at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')

            #determine the magnitudes, errors and distances to the objects
            #prepare the apertures
            positions = [xx,yy]
            apertures = CircularAperture(positions, r = rad)

            #get the photometry for the stars
            rawflux = aperture_photometry(img, apertures)

            #get the background	
            bkg_mean = median
            bkg_sum = bkg_mean*(np.pi*rad**2)
            
            #print("rad",rad,"bkg_mean",bkg_mean,"bkg_sum",bkg_sum,"aperture_photometry",aperture_photometry,"mflx",mflx)
            #get the star flux and error & mag and error
            
            
            #comentei a retirada de backgroun
            flx = rawflux['aperture_sum']#-bkg_sum
            flx_er = np.sqrt(rawflux['aperture_sum'])
            mag = 25.-2.5*np.log10(flx)
            mag_er = (2.5/np.log(10.))*(flx_er/flx)

            #print('Getting reference stars for the subtraction at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')
            #star selecting stars that are not near any other bright stars	
            output = open(cdedir+'refstars.txt', 'w')
            cnt = 0
            itr = 0
            x = xx
            y = yy

            while (cnt < nrstars) and (itr < len(xx)):
                #select a random object
                jj = random.randint(0,len(x)-1)
                if (mag_er[jj] > 0) and (mag_er[jj] < 0.02):
                    #get the nearest neightbors in 3 pix
                    dist = np.sqrt((x[jj]-x)**2+(y[jj]-y)**2)
                    idx = np.where(dist < 3)
                    idxs = idx[0]
                    #assuming the star is alone and it is not near an edge
                    if (len(idxs) == 1):
                        output.write("%4d %4d\n" % (x[jj],y[jj]))
                        cnt = cnt+1
                    else:
                        if (len(idxs) > 0):
                            #check the magnitudes in case the neighbors are just faint stars
                            dmag = mag[jj]-mag[idxs]
                            cdmag = dmag[np.where(dmag != 0)]
                            chk = np.where(cdmag > 0)
                            if (len(chk[0]) == 0):
                                output.write("%4d %4d\n" % (x[jj],y[jj]))
                                cnt = cnt+1
                x = np.delete(x,jj)
                y = np.delete(y,jj)
                itr = itr+1
            nrstars = cnt #update the number of stars to use, just in case the maximum wasn't found but we ran out of stars
            output.close()
            imglist.close()


            #write the parameter file now that we have the stars
            output = open(cdedir+'parms.txt', 'w')
            output.write("%1d %1d %1d %4d\n" % (stmp, krnl, ordr, nrstars))
            output.close()

            output = open(cdedir+'ref.txt', 'w')
            output.write("ref.fits\n")
            output.close()

            output = open(cdedir+'img.txt', 'w')
            output.write("img.fits\n")
            output.close()


            #do the differencing!
            #print('Now starting the subtraction at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')
#             os.system('gcc sub_teste.c -lcfitsio -lm -lcurl')
#             os.system('./a.out')
            dodiff = os.system('./a.out')
#             os.system('mv dimg.fits '+difdir+'/'+finnme)
            mvdiff = os.system('mv dimg.fits '+difdir+'/'+finnme)

            #get the photometry from the differenced image
            #print('Now starting the photometry at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')

            #read in the image
            diflist = fits.open(difdir+'/'+finnme)
            iheader = diflist[0].header #get the header info
            dif = diflist[0].data #get the image info

            #print('Getting fluxes from the differenced file at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')
            
            #determine the magnitudes, errors and distances to the objects
            #prepare the apertures
            positions = [xx,yy]
            apertures = CircularAperture(positions, r = rad)

            #get the photometry for the stars
            rawflux = aperture_photometry(dif, apertures)
            #print(rawflux)

            #get the background	
            #### ADICIONEI ESSE SIGMA CLIP
            mean, median, std = sigma_clipped_stats(dif, sigma = 3.0, iters = 5)
            ##########
            bkg_mean = median
            bkg_sum = bkg_mean*(np.pi*rad**2)
            #print(rawflux,"bkg sum eh",bkg_sum)

            #get the star flux and error & mag and error
            
            flx = rawflux['aperture_sum']#-bkg_sum
            flx_er = np.sqrt(np.abs(rawflux['aperture_sum']))
            mag = 25.-2.5*np.log10(flx/exp_time+mflx/expm_time)
            mag_er = (2.5/np.log(10.))*(np.sqrt((flx_er/exp_time)**2+(mflx_er/expm_time)**2)/(flx/exp_time+mflx/expm_time))
            diflist.close()

            #print the flux information to the data file
            nme = finnme.split('.')
            output = open(difdir+nme[0]+'.flux', 'w')
            for jj in range(0, len(xx)):
                output.write(str(ids[jj])+','+str(xm[jj])+','+str(ym[jj])+','+str(jd)+','+str(flx[jj])+','+str(flx_er[jj])+','+str(mag[jj])+','+str(mag_er[jj])+','+season+'\n')
            output.close()
            print('Moving to the next file at '+strftime("%a, %d %b %Y %H:%M:%S")+'.')
print('All done at '+strftime("%a, %d %b %Y %H:%M:%S")+'. See ya later alligator!')


kplr2009114174833_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:58:11.
kplr2009114204835_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:58:20.
kplr2009115002613_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:58:28.
kplr2009115053616_ffi-cal_sadxx.fits


/home/usuario/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:138: RuntimeWarning: invalid value encountered in log10


Moving to the next file at Tue, 19 Nov 2019 10:58:36.
kplr2009115080620_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:58:44.
kplr2009115131122_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:58:52.
kplr2009115173611_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:00.
kplr2009116035924_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:08.
kplr2010111125026_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:17.
kplr2010140101631_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:25.
kplr2010174164113_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:33.
kplr2011116104002_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:41.
kplr2011145152723_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:49.
kplr2011177110110_ffi-cal_sadxx.fits
Moving to the next file at Tue, 19 Nov 2019 10:59:57.
kplr2012121122500_ffi-cal_sadxx.fits